In [17]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
df=pd.read_csv('all_season_details.csv' ,low_memory= True)

C:\Users\govinda prasad\AppData\Local\Temp\ipykernel_31500\2897799502.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('all_season_details.csv' ,low_memory= True)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\IPython\core\ultratb.py", line 1457, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\IPython\core\ultratb.py", line 1348, in structured_traceback
    return VerboseTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\IPython\core\ultratb.py", line 1195, in structured_traceback
    formatted_exception = self.format_exception_as_a_whole(etype, evalue, etb, number_of_lines_of_context,
                  

In [ ]:
df.drop(['comment_id','shortText','text','preText','postText'],axis=1,inplace=True)


In [164]:
df.loc[(df['match_id'].isin([1359496,1359538])) & (df['season'].isnull()), 'season'] = 2023


In [165]:
df.drop_duplicates(inplace=True)


In [166]:
new_df = df[['season', 'match_id', 'home_team', 'away_team', 'current_innings',
                      'innings_id', 'over', 'ball', 'runs', 'wicket_id', 
                      'wkt_batsman_name', 'wkt_batsman_runs']].copy()

new_df['cumulative_runs'] = new_df.groupby(['match_id', 'current_innings'])['runs'].cumsum()

new_df['run_rate'] = new_df['cumulative_runs'] / (new_df['over'] + 0.1)

new_df['wickets_lost'] = new_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                                 .transform(lambda x: (x >= 0).cumsum())
new_df.fillna(0, inplace=True)


In [168]:
unique_home_teams = new_df['home_team'].unique().tolist()

home_teams_df = pd.DataFrame(unique_home_teams, columns=['Teams'])

print(home_teams_df)


    Teams
0      GT
1    PBKS
2     LSG
3     SRH
4     RCB
5     CSK
6      DC
7      RR
8     KKR
9      MI
10   KXIP
11    RPS
12     GL
13    PWI
14  Kochi


In [174]:
# Create a dictionary for mapping
team_mapping = {'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3,'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
                'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11,'GL': 12, 'PWI': 13, 'KOCHI': 14}

# Apply the mapping to 'home_team' and 'away_team'
new_df['home_team_encoded'] = new_df['home_team'].map(team_mapping)
new_df['away_team_encoded'] = new_df['away_team'].map(team_mapping)

new_df['current_innings'] = new_df['current_innings'].map(team_mapping)
new_df.fillna(0, inplace=True)


In [176]:
X = new_df[['home_team_encoded','away_team_encoded','current_innings','over', 'ball', 'run_rate','wickets_lost']]
y = new_df['cumulative_runs']

In [178]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [180]:
X_train

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
199033,13.0,8.0,13.0,12,4,5.537190,3
237647,6.0,3.0,6.0,17,1,8.596491,2
10462,0.0,6.0,6.0,1,5,4.545455,1
39476,7.0,4.0,7.0,8,4,8.518519,0
137929,3.0,5.0,3.0,9,2,6.153846,2
...,...,...,...,...,...,...,...
119880,8.0,3.0,3.0,10,6,7.029703,3
103695,3.0,9.0,3.0,12,4,8.016529,1
131933,10.0,4.0,4.0,6,1,8.196721,2
146868,8.0,3.0,3.0,18,6,7.182320,5


In [182]:
rf_model=RandomForestRegressor()
rf_model.fit(X_train,y_train)

y_rf_simple_pred=rf_model.predict(X_test)
rf_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

print('MSE score: ',rf_mse)
print('R2 score: ',rf_r2)

MSE score:  0.05560748093176666
R2 score:  0.9999777188978023


In [183]:
new_data = pd.DataFrame({
    'home_team_encoded': [1],
    'away_team_encoded': [5],
    'current_innings':[1],
    'over': [12],  
    'ball': [2],      
    'run_rate': [10],        
    'wickets_lost': [2]          
})

In [184]:
predict_runs=rf_model.predict(new_data)
print(f"Predicted Cumulative Runs: {predict_runs[0]}")

Predicted Cumulative Runs: 121.0


In [188]:
print('Linear  regression')

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

pred = lr_model.predict(X_test)

lr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',lr_mse)
lr_r2=r2_score(y_test,pred)
print('R2 score: ',lr_r2)

Linear  regression
MSE score:  266.7656880261982
R2 score:  0.8931109005809659


In [190]:
print('Polynomial  regression')

poly = PolynomialFeatures(degree=2) 
X_polya = poly.fit_transform(X_train)
X_polyb=poly.fit_transform(X_test)


poly_model = LinearRegression()
poly_model.fit(X_polya, y_train)


pred = poly_model.predict(X_polyb)

pr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',pr_mse)

pr_r2=r2_score(y_test,pred)
print('R2 score: ',pr_r2)


Polynomial  regression
MSE score:  4.04188542156602e-24
R2 score:  1.0


In [192]:
print('Decision Tree  regression')

from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=3)
tree_model.fit(X_train, y_train)

pred = tree_model.predict(X_test)

tree_mse=mean_squared_error(y_test,pred)
print('MSE score: ',tree_mse)

tree_r2=r2_score(y_test,pred)
print('R2 score: ',tree_r2)

Decision Tree  regression
MSE score:  327.7749015281932
R2 score:  0.8686654033517592


In [194]:
predicted_runs_tree = tree_model.predict(new_data)
predicted_runs_tree

array([115.09278835])

In [196]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("Gradient Boosting Regressor:")
print("MSE score: ",mse_gb)
print("R2 score: ",r2_gb)

Gradient Boosting Regressor:
MSE score:  3.4692460767645668
R2 score:  0.9986099239690377


In [197]:
from xgboost import XGBRegressor


xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost Regressor:")
print("MSE score: ",mse_xgb)
print("R2 score: ",r2_xgb)

XGBoost Regressor:
MSE score:  1.4133610932762821
R2 score:  0.999433696269989


In [198]:
model_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Linear Regression', 'Polynomial Regression', 'Decision Tree','Gradient Boosting','XGBoost Regressor'],
    'MSE': [rf_mse, lr_mse, pr_mse, tree_mse,mse_gb,mse_xgb],
    'R² Score': [rf_r2, lr_r2, pr_r2, tree_r2,r2_gb,r2_xgb]
})

# Display the comparison
print(model_comparison)

                   Model           MSE  R² Score
0          Random Forest  5.560748e-02  0.999978
1      Linear Regression  2.667657e+02  0.893111
2  Polynomial Regression  4.041885e-24  1.000000
3          Decision Tree  3.277749e+02  0.868665
4      Gradient Boosting  3.469246e+00  0.998610
5      XGBoost Regressor  1.413361e+00  0.999434


In [203]:
X_sample = X_train.sample(frac=0.1, random_state=42)  # Take 10% of the data
y_sample = y_train.sample(frac=0.1, random_state=42)


In [205]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Define the parameter grid for RandomizedSearchCV
# Define parameter grid for DecisionTreeRegressor
param_distributions_tree = {
    'max_depth': [3, 5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'splitter': ['best', 'random']
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=tree_model,
    param_distributions=param_distributions_tree,
    n_iter=100,
    cv=3,
    n_jobs=1,  # Limit to 1 process
    verbose=1
)

# Fit the model with the training data
random_search.fit(X_sample, y_sample)

# Get the best parameters and print them
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")




Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found: {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'criterion': 'friedman_mse'}


C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
237 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
102 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\govinda prasad\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", li

In [207]:
# Get the best model and make predictions
best_tree_model = random_search.best_estimator_
tree_predictions = best_tree_model.predict(X_test)

# Evaluate the model
tree_mse = mean_squared_error(y_test, tree_predictions)  # Assuming y_test is your test target
tree_r2 = r2_score(y_test, tree_predictions)

# Print the evaluation metrics
print(f"Random Forest Regressor - MSE: {tree_mse:.4f}, R2 Score: {tree_r2:.4f}")


Random Forest Regressor - MSE: 3.1861, R2 Score: 0.9987


In [209]:
tree_mse = mean_squared_error(y_test, tree_predictions) # Assuming y_test is your test target
tree_r2 = r2_score(y_test, tree_predictions)
print(f"Random Forest Regressor - MSE: {tree_mse:.4f}, R2 Score: {tree_r2:.4f}")
     

Random Forest Regressor - MSE: 3.1861, R2 Score: 0.9987


In [211]:
predicted_runs_tree = tree_model.predict(new_data)
predicted_runs_tree

array([115.09278835])